In [1]:
import tweepy
from textblob import TextBlob
from wordcloud import WordCloud
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

In [2]:
config=pd.read_csv("./config.csv")

In [3]:
ApiKey=config['ApiKey'][0]
ApiSecret=config['ApiSecret'][0]
ApiAccessToken=config['ApiAccessToken'][0]
ApiAccessTokenSecret=config['ApiAccessTokenSecret'][0]

In [4]:
#creating authentication object
auth = tweepy.OAuthHandler(ApiKey, ApiSecret)

#set the access token and access token secret
auth.set_access_token(ApiAccessToken,ApiAccessTokenSecret)

#create the API object while passing in the auth function
api=tweepy.API(auth, wait_on_rate_limit=True)

In [5]:
tag='depressed'

In [6]:
tweets= api.search_tweets(q=tag,lang='en',count=100)

In [7]:
i=0
for tweet in tweets:
    print('('+str(i)+'.) '+tweet.text+'\n')
    i+=1

(0.) RT @lilsproutboi: Tony immediately knowing that Yuno’s sadge farming judging by his tone 😭

Tony: “Blau, u logging back in? Yuno’s getting…

(1.) RT @Rich_Cooper: If masculinity is so toxic, then kids growing up without dads would be better off than those with them.

But they aren't,…

(2.) RT @Mumtaz_Bano17: TAHAJJUD REMINDER!
People don't skip TAHAJJUD, especially this month &amp; next. Do loads of duas. Persuade Allah to change…

(3.) RT @mxrvssa: Fuckin a dead beat is not a flex. Fuckin somebody knowing that they have a baby elsewhere and the mother is depressed and poss…

(4.) When you are depressed but couldn't smoke a cigarette 😥

(5.) RT @Mumtaz_Bano17: TAHAJJUD REMINDER!
People don't skip TAHAJJUD, especially this month &amp; next. Do loads of duas. Persuade Allah to change…

(6.) RT @mxrvssa: Fuckin a dead beat is not a flex. Fuckin somebody knowing that they have a baby elsewhere and the mother is depressed and poss…

(7.) RT @dxzysLIVE: @notch @C418 notch tweeting from

In [8]:
#create a dataframe with a column called tweets

df=pd.DataFrame([tweet.text for tweet in tweets], columns=['Tweets'])

In [9]:
#cleaning data
def cleanTxt(text):
    text = text.lower()
    text = re.sub(r'@[A-Za-z0-9]+', '',text)
    text = re.sub(r'#','',text)
    text = re.sub(r'RT[\s]+','',text)
    text = re.sub(r'https?:\/\/S+','',text)
    text = re.sub(r'<Emoji:.>','',text)
    text = re.sub(r"(@\[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", text)
    return text

df['Tweets']=df['Tweets'].apply(cleanTxt)
df

,Tweets
0,tony immediately knowing that yunos sadge fa...
1,cooper if masculinity is so toxic then kids g...
2,bano17 tahajjud reminderpeople dont skip taha...
3,fuckin a dead beat is not a flex fuckin some...
4,when you are depressed but couldnt smoke a cig...
...,...
95,from angry 247 to depressed 247 karena dibull...
96,story threada beautiful woman in an expensiv...
97,depressed guy seen too much looking at compu...
98,fuckin a dead beat is not a flex fuckin some...


In [10]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords
set(stopwords.words('english'))
from nltk.tokenize import word_tokenize
def removeStopWords(text):
    text_token = word_tokenize(text)
    token_without_sw = [word for word in text_token if not word in stopwords.words()]
    filtered_text = " ".join(token_without_sw)
    print(filtered_text)
    return filtered_text
df['Tweets']=df['Tweets'].apply(removeStopWords)
df

In [13]:
def checksubjectivity(text):
    return TextBlob(text).sentiment.subjectivity
def checkpolarity(text):
    return TextBlob(text).sentiment.polarity
df['polarity'] = df['Tweets'].apply(checkpolarity)
df

,Tweets,polarity
0,women dont be caring if they man depressed o...,-0.250000
1,pernah tak kau busykan diri kau untuk hilang...,-0.500000
2,only poor people think this while tons of ri...,-0.181250
3,quran verse for when youre feelingupset 315...,0.000000
4,deku depressed traumatized constantly on the...,-0.033333
...,...,...
95,02 pet,0.000000
96,casteist slurs are not satirestudents at jai...,0.000000
97,spiders ive been obsessed with soggy depresse...,-0.500000
98,blue storks are never depressed,0.000000


In [1]:
words = ' '.join(t for t in df['Tweets'])
wordcloud = WordCloud(width = 500, height = 300, random_state = 21, max_font_size = 119).generate(words)
plt.imshow(wordcloud, interpolation = "bilinear")
plt.axis("off")
plt.show

NameError: name 'df' is not defined

In [ ]:
df["Sentiment"]=[None for i in range(len(df))]
for i in range(len(df)):
    s = TextBlob(df["Tweets"].iloc[i]).sentiment
    print(s)
    if(s[0]>=0):
        df["Sentiment"].iloc[i] = 0 #not-depressed
    else:
        df["Sentiment"].iloc[i] = 1 #depressed

In [2]:
print("not depressed="+str(len(df[df["Sentiment"]==0])))
print("depressed="+str(len(df[df["Sentiment"]==1])))

NameError: name 'df' is not defined

In [ ]:
df

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(df['Tweets'],df['Sentiment'],test_size=0.2,random_state=84)

In [ ]:
len(X_train)

In [ ]:
len(X_test)

In [ ]:
#VECTORIZATION
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
cv = TfidfVectorizer(ngram_range=(1,2))
X_vec_train = cv.fit_transform(X_train).toarray()
X_vec_train

In [ ]:
print(cv.get_feature_names())

In [ ]:
X_vec_test = cv.transform(X_test).toarray()

In [ ]:
X_vec_test

In [ ]:
# Multinomial Naive Bayes
from sklearn.naive_bayes import MultinomialNB

In [ ]:
mn_obj = MultinomialNB()

In [ ]:
len(X_vec_train),len(Y_train)
Y_train = Y_train.astype('int64')
X_vec_train.dtype,Y_train.dtype
mn_obj.fit(X_vec_train,Y_train)

In [ ]:
Y_prediction = mn_obj.predict(X_vec_test)

In [ ]:
X_test

In [ ]:
#1 means depressed
Y_prediction

In [ ]:
from sklearn import metrics
Y_test = Y_test.astype('int64')
accuracy = metrics.accuracy_score(Y_test, Y_prediction)

In [ ]:
accuracy